In [ ]:
import pandas as pd
import json

In [21]:
# Load toolbench bias queries
with open("toolbench_bias_queries.json", "r") as input_file, open("openai_format.json", "w") as output_file:
    toolbench_bias_queries = json.load(input_file)
    
    count = 0
    types = set()

    clusters = []

    # Decisions for converting to OpenAI format:
    # 1. Function Name will be: f{tool_name}.{api_name} (both converted to snake_case) 
    # 2. Parameters will be "parameters": {"type":"object", "properties": {"{param_name}": {"type": "{type}", "description": "{description}"}}, required: ["{param_name}"]}


    # For Types
    # ENUM, string, STRING --> string
    # DATE (YYYY-MM-DD) --> string
    # BOOLEAN --> boolean
    # NUMBER --> float
    type_mapping = {
        "ENUM": "string",
        "DATE (YYYY-MM-DD)": "string",
        "BOOLEAN": "boolean",
        "NUMBER": "float",
        "STRING": "string",
        "string": "string",
    }

    count = 1
    for idx in range(0, len(toolbench_bias_queries), 500):
        apis = toolbench_bias_queries[idx]['api_list']

        openai_apis = []

        
        for api in apis:

            # Convert to snake_case
            api_name = api['api_name'].lower().replace(" ", "_")
            tool_name = api['tool_name'].lower().replace(" ", "_")
            function_name = f"{tool_name}.{api_name}"

            # Convert to OpenAI format
            required = []
            properties = {}
            for param in api['required_parameters']:
                types.add(param['type'])
                properties[param['name']] = {
                    "type": type_mapping[param['type']],
                    "description": param['description']
                }
                required.append(param['name'])

            for param in api['optional_parameters']:
                types.add(param['type'])
                properties[param['name']] = {
                    "type": type_mapping[param['type']],
                    "description": param['description']
                }

            openai_apis.append({
                "name": function_name,
                "parameters": {
                    "type": "object",
                    "properties": properties,
                    "required": required
                }})
        clusters.append({"cluster_id": count, "tools": openai_apis})
        count += 1
    
    print(types)

    json.dump(clusters, output_file)

    



{'BOOLEAN', 'DATE (YYYY-MM-DD)', 'NUMBER', 'ENUM', 'string', 'STRING'}
